In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# to load large .mat files
import mat73
import random
from net import EEG_CNN

### Train the CNN

In [ ]:
eeg_topos = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/eeg_topos_2TeD.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

In [ ]:
topo_idx = random.randint(0,eeg_topos.shape[0]-1)
topo = eeg_topos[topo_idx,:,:]
plt.contourf(topo, cmap=cm.get_cmap('viridis'))
cbar = plt.colorbar()
plt.draw()
plt.title('Topography for eeg signal: {}'.format(topo_idx+1))

In [ ]:
# load previous simulation
sources = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/sources_2TeD.npy')
eeg = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/eeg_2TeD.npy')

fwd = solve_forward()
sim = Simulation(fwd=fwd, source_data=sources, eeg_data=eeg)
# fwd = solve_forward()
# sim = Simulation(fwd=fwd)
# sim.simulate(n_samples=100)

In [ ]:
eegcnn = EEG_CNN(sim=sim, eeg_topographies=eeg_topos)

In [ ]:
# eegcnn.build_model()
eegcnn.load_nn('/media/thanos/Elements/thanos/nn_trained/cnn_model.h5')

In [ ]:
eegcnn.fit(epochs=380, patience=70, batch_size=128)

In [ ]:
eegcnn.save_nn('../cnn_model_2.h5')

### Evaluate the CNN

In [ ]:
eegcnn.load_nn('/media/thanos/Elements/thanos/nn_trained/cnn_model.h5')

In [ ]:
eeg_topos_eval = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/eval_100/eeg_topos_eval_100.mat')['eeg_topos']
eeg_topos_eval = eeg_topos_eval.transpose(2, 0, 1)

sources_eval = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/eval_100/sources.npy')


In [ ]:
predicted_sources = eegcnn.predict(eeg=eeg_topos_eval).T

In [ ]:
np.save('../../Downloads/predicted_sources.npy',predicted_sources)